### Agents in LLMs

In [1]:
import json

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.tools import StructuredTool
from langchain_core.pydantic_v1 import BaseModel
from langchain.utilities import PythonREPL
from common_functions import get_ollama

llm = get_ollama()

In [2]:
tools = load_tools([
    'requests_all', 'llm-math', 'openweathermap-api',
    'google-serper', 'ddg-search'  # DuckDuckGo search tool
], llm=llm)  # 'wikipedia', 'google-search'

python_repl = PythonREPL()
class ArgsSchema(BaseModel):
    command: str

tools.append(StructuredTool(
	name='Python repl',
	func=python_repl.run,
	description='useful when you need to run python code. ',
	args_schema=ArgsSchema,
))

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # uses Chain-of-Thought
	verbose=True,
	handle_parsing_errors=True,
)

def ask_query(query):
	response = agent.invoke({'input': query})
	if isinstance(response, dict):
		response = response.get('output')
	response = response.strip()

	if '{' in response:
		# get text from first { to last }
		response = response[response.index('{'):response.rindex('}')+1]
	if response[0] == '{':
		response_json = json.loads(response)
		answer = response_json.get('action_input')
		return answer

	if '<|eot_id|>' in response:  # get part before that
		response = response[:response.index('<|eot_id|>')]
	return response

/home/praneeth/Desktop/AI_projects/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [3]:
ask_query('Write and execute python code to print hello world')



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


Thought:
Action:
```
{
   "action": "Python repl",
   "action_input": "print('hello world')"
}
```

Observation: hello world

Thought:I'm glad to continue!

Thought:
The human wants me to write and execute python code to print "hello world". We've already done that, and it worked perfectly!
Action:
```{
   "action": "Final Answer", 
   "action_input": "No further action required. The task is complete."
}```


> Finished chain.


'No further action required. The task is complete.'

In [4]:
ask_query('Who is the current President of France in May 2024?')



> Entering new AgentExecutor chain...
Thought:
The human is asking about the current President of France.

Action:
```
{
   "action": "Final Answer",
   "action_input": "Emmanuel Macron"
}
```



> Finished chain.


'Emmanuel Macron'

In [5]:
ask_query('What is weather of the capital of France?')



> Entering new AgentExecutor chain...
Thought:
Action:
```
{
   "action": "open_weather_map",
   "action_input": "Paris,FR"
}
```

Observation: In Paris,FR, the current weather is as follows:
Detailed status: clear sky
Wind speed: 3.09 m/s, direction: 50°
Humidity: 77%
Temperature: 
  - Current: 16.78°C
  - High: 19.03°C
  - Low: 15.0°C
  - Feels like: 16.52°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought:Thought:
Action: 
```{
    "action": "Final Answer", 
    "action_input": "The current weather in Paris, FR is clear sky with a temperature of 16.78°C."
}```


> Finished chain.


'The current weather in Paris, FR is clear sky with a temperature of 16.78°C.'

In [6]:
ask_query('On which date was George Washington born?')



> Entering new AgentExecutor chain...
Thought: I know that George Washington is one of the most well-known American founding fathers.

Action:
```
{
  "action": "Final Answer",
  "action_input": "February 22, 1732"
}
```



> Finished chain.


'February 22, 1732'

## For more projects, open [README.md](/README.md)

___

In [ ]:
# Unused code

# from langchain import hub# print('Pulling prompt...', end=' ', flush=True)
# prompt = hub.pull('hwchase17/structured-chat-agent')
# print('done')
# agent = create_structured_chat_agent(tools, llm, prompt=prompt)
# agent_executor = AgentExecutor(
#     agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
# )


# Memorizer tool
# from langchain.tools import BaseTool, tool, Tool
# memory_filepath = os.path.join(datasets_dir, 'memory.txt')

# @tool
# def get_memory() -> list[str]:
# 	'Reads entire memory from file. Use this when user asks to remember the things' \
# 		'Inputs: None. Outputs: List of strings'
# 	if not os.path.exists(memory_filepath):
# 		return None
# 	with open(memory_filepath) as f:
# 		memory = f.readlines()
# 	return [line.strip() for line in memory]

# @tool
# def set_memory(memory: list[str]):
# 	'Rewrites entire memory to file. Use this when user asks to remember the things' \
# 		'Inputs: List of strings. Outputs: None'
# 	with open(memory_filepath, 'w') as f:
# 		f.writelines(memory)
# 	print('set_memory: rewrote', len(memory), 'lines')

# tools = [set_memory, get_memory] + tools

# ask_query('Remember I have put my keys on the table')
# ask_query('Where are my keys?')